In [ ]:
# imports and link xml
import re
from pprint import pprint
import xml.etree.ElementTree as ET
tree = ET.parse('session/Kismet-20191011-18-31-55-1.netxml')
root = tree.getroot()

In [ ]:
StationList = [] # MAC, [[min GPS],[max GPS]], [[first seen],[last seen]], device type, encryption, ESSID

In [3]:
def getDeviceType(bssid): # find vendor based off MAC address
    file = open('mac-vendors.txt', 'r', encoding="utf8")
    raw=[]
    mac_dict={}
    for line in file:
        raw.append(re.split(r'\t+', line.strip()))
    for pair in raw:
        mac_dict[pair[0]]= pair[1]
    return mac_dict.get(bssid[:8])

In [4]:
def appendArray(net, child): # parse xml and create array data
    if (child.tag == net) :
        bssid, essid = "", ""
        gps, time, encryption = [], [], []
        if (net=="wireless-network"): # network xml tag
            scan= 'BSSID'
        else:
            scan= 'client-mac'        # client xml tag
        for bssid_tag in child.findall(scan):
            bssid = bssid_tag.text
        for gps_tag in child.findall("gps-info"):
            gps= [[gps_tag[0].text, gps_tag[1].text],[gps_tag[4].text, gps_tag[5].text]]
        time = [child.attrib['first-time'],child.attrib['last-time']]
        for ssid_tag in child.findall('SSID'):
            for essid_tag in ssid_tag.findall('essid'):
                essid= essid_tag.text
            for info_tag in ssid_tag :
                if (info_tag.tag == "encryption"):
                    encryption.append(info_tag.text)
        StationList.append([bssid,gps,time,getDeviceType(bssid),encryption,essid]) 

In [5]:
for child in root:
    appendArray("wireless-network",child) #networks
    for subchild in child:
        appendArray("wireless-client", subchild) #clients

In [6]:
import pandas as pd
pd.DataFrame(StationList, columns=["BSSID", "GPS","Dates","Manufacturer","Encryption","ESSID"])

,BSSID,GPS,Dates,Manufacturer,Encryption,ESSID
0,00:02:6F:22:86:21,"[[34.195740, -118.340050], [34.195740, -118.34...","[Fri Oct 11 18:36:36 2019, Fri Oct 11 18:36:36...","Senao International Co., Ltd.",[None],Downtown-Burbank
1,00:02:6F:22:86:21,"[[34.195740, -118.340050], [34.195740, -118.34...","[Fri Oct 11 18:36:36 2019, Fri Oct 11 18:36:36...","Senao International Co., Ltd.",[],
2,00:0D:97:00:59:05,"[[34.191360, -118.341759], [34.191360, -118.34...","[Fri Oct 11 18:38:10 2019, Fri Oct 11 18:38:10...",ABB Inc./Tropos,[],
3,00:0D:97:00:59:05,"[[34.191360, -118.341759], [34.191360, -118.34...","[Fri Oct 11 18:38:10 2019, Fri Oct 11 18:38:10...",ABB Inc./Tropos,[],
4,00:0D:97:00:5A:92,"[[34.190983, -118.341637], [34.191032, -118.34...","[Fri Oct 11 18:40:30 2019, Fri Oct 11 18:41:52...",ABB Inc./Tropos,[None],BWP Free WiFi
5,00:0D:97:00:5A:92,"[[34.190983, -118.341637], [34.191032, -118.34...","[Fri Oct 11 18:40:30 2019, Fri Oct 11 18:41:52...",ABB Inc./Tropos,[],
6,00:0D:97:00:5B:24,"[[34.191032, -118.341621], [34.191032, -118.34...","[Fri Oct 11 18:40:30 2019, Fri Oct 11 18:40:30...",ABB Inc./Tropos,[WEP],None
7,00:0D:97:00:5B:24,"[[34.191032, -118.341621], [34.191032, -118.34...","[Fri Oct 11 18:40:30 2019, Fri Oct 11 18:40:30...",ABB Inc./Tropos,[],
8,00:0D:97:00:5B:30,"[[34.190819, -118.341728], [34.191090, -118.34...","[Fri Oct 11 18:38:39 2019, Fri Oct 11 18:46:24...",ABB Inc./Tropos,"[WEP, None]",BWP Free WiFi
9,00:0D:97:00:5B:30,"[[34.190819, -118.341728], [34.191090, -118.34...","[Fri Oct 11 18:38:39 2019, Fri Oct 11 18:46:24...",ABB Inc./Tropos,[],
